In [7]:
import tensorflow as tf

# Allow GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU available.")

# Initialize GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

1 Physical GPUs, 1 Logical GPUs
Num GPUs Available:  1


In [8]:
# We get the test set for landmarks and for masks.

import os
import cv2
import numpy as np
import tensorflow as tf

# Rutas a las carpetas de entrenamiento, prueba y validación para imágenes y máscaras
test_frames_dir = "./Frames/TEST"
test_Landmarks_dir = "./Landmarks/Landmarks/TEST"
test_masks_dir = "./Mascaras/Mask/TEST"

# Funciones para cargar y preprocesar imagen y máscara y convertirlo a solo un canal
def load_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image / 255.0
    return image

def load_landmarks(file_path):
    images = []
    for filename in os.listdir(file_path):
        image=cv2.imread(os.path.join(file_path, filename))
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(grayscale_image)

    stacked_image = np.stack(images, axis=-1)  # axis=-1 indica la dimensión de los canales
    stacked_image = stacked_image / 255.0
    return stacked_image

test_image_files = [os.path.join(test_frames_dir, filename) for filename in os.listdir(test_frames_dir)]
test_Landmarks_files = [os.path.join(test_Landmarks_dir, filename) for filename in os.listdir(test_Landmarks_dir)]
test_mask_files = [os.path.join(test_masks_dir, filename) for filename in os.listdir(test_masks_dir)]

test_images = [load_image(file) for file in test_image_files]
test_Landmarks = [load_landmarks(file) for file in test_Landmarks_files]
test_masks = [load_image(file) for file in test_mask_files]

test_images = np.array(test_images)
test_Landmarks = np.array(test_Landmarks)
test_masks = np.array(test_masks)

In [9]:
BATCH_SIZE = 16

with tf.device('CPU'):
    test_dataset_masks = tf.data.Dataset.from_tensor_slices((test_images, test_masks))
    test_dataset_masks = test_dataset_masks.batch(BATCH_SIZE)

    test_dataset_Landmarks = tf.data.Dataset.from_tensor_slices((test_images, test_Landmarks))
    test_dataset_Landmarks = test_dataset_Landmarks.batch(BATCH_SIZE)

In [10]:
# We get the model for the masks and the model for the landmarks.
